1. Import Libraries

In [24]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge geocoder --yes
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


2. Get Dataset

In [143]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Mumbai").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html5lib')

In [144]:
# create a list to store neighborhood data
TouristPlaceList = []
# append the data into the list
#for row in soup.find_all("div",class_="toc")[0].findAll("li"):
#    TouristPlaceList.append(row.text)
#for row in soup.find_all("ul")[2].findAll("li"):
#    TouristPlaceList.append(row.text)
#TouristPlaceList 2-14
for i in range(2,14):
    for row in soup.find_all("ul")[i].findAll("li"):
        TouristPlaceList.append(row.text)
TouristPlaceList = pd.DataFrame({"TouristPlace": TouristPlaceList})
TouristPlaceList.head()

,TouristPlace
0,Adlabs Imagica
1,Andheri Sports Complex
2,B.P.T. Ground
3,Bandra Kurla Complex Ground
4,Brabourne Stadium


In [146]:
TouristPlaceList.shape

(162, 1)

Get Geo Coordinates

In [151]:
def get_latlng(TouristPlace):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(TouristPlace))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [152]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(TouristPlace) for TouristPlace in TouristPlaceList["TouristPlace"].tolist() ]

In [154]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [155]:
# merge the coordinates into the original dataframe
TouristPlaceList['Latitude'] = df_coords['Latitude']
TouristPlaceList['Longitude'] = df_coords['Longitude']
# check the neighborhoods and the coordinates
print(TouristPlaceList.shape)
TouristPlaceList

(162, 3)


,TouristPlace,Latitude,Longitude
0,Adlabs Imagica,18.768290,73.282610
1,Andheri Sports Complex,19.128920,72.836290
2,B.P.T. Ground,19.017470,72.819200
3,Bandra Kurla Complex Ground,19.066840,72.861930
4,Brabourne Stadium,18.932580,72.823470
5,Colaba Woods,18.911520,72.819120
6,Cooperage Ground,18.924440,72.829580
7,Cross Maidan,18.939900,72.829850
8,Dadaji Kondadev Stadium,19.193170,72.980030
9,DY Patil Stadium,19.041350,73.027080


#### Map of Mumbai

In [156]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [167]:
# create map of Toronto using latitude and longitude values
map_mi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, TouristPlace in zip(TouristPlaceList['Latitude'], TouristPlaceList['Longitude'], TouristPlaceList['TouristPlace']):
    label = '{}'.format(TouristPlace)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mi)  
    
map_mi

In [160]:
# save the map as HTML file
map_mi.save('map_mi_1.html')

Foursquare API to explore the neighborhoods of the Tourist places